1. data2 로드를 통한 데이터 프레임 완성
2. 한글에 맞게 형태소 분석기를 이용해서 단어 토큰화 TF-IDF 계산
3. 코사인 유사도를 기반으로 영화 3종 추천 -올드보이

In [ ]:
import pandas as pd
import numpy as np
! pip install konlpy
from konlpy.tag import Okt

In [2]:
data = pd.read_csv('data2.csv')
data.head(), data.shape

(     name  genre                                            content
 0    올드보이    스릴러  술 좋아하고 떠들기 좋아하는 오.대.수. 본인의 이름풀이를 오늘만 대충 수습하며 살...
 1     노트북    로맨스  17살, ‘노아’는 밝고 순수한 ‘앨리’를 보고 첫눈에 반한다. 빠른 속도로 서로에...
 2  미션임파서블     액션  도난당한 핵무기를 되찾아 오는 위험천만한 작전 중, 에단 헌트(톰 크루즈)는 임무를...
 3  니모를찾아서  애니메이션  “디즈니`픽사 최고의 애니메이션이 3D로 다시 찾아온다!” 호기심 가득한 아기 물고...
 4      탑건     액션  해군 최신 전투기 F-14기를 모는 젊은 조종사 매버릭 대위(톰 크루즈)는 최고의 ..., (18, 3))

In [3]:
okt = Okt() #
data_l = [' '.join(okt.morphs(i)) for i in data['content']]
data_l[0]

"술 좋아하고 떠들기 좋아하는 오 . 대 . 수 . 본인 의 이름 풀이 를 오늘 만 대충 수습 하며 살자 ' 라고 이죽거리는 이 남자 는 아내 와 어린 딸아이 를 가진 지극히 평범한 샐러리맨 이다 . 어느 날 , 술 이 거나 하게 취해 집 에 돌아가는 길 에 존재 를 알 수 없는 누군가 에게 납치 , 사설 감금 방 에 갇히게 되는데 . 언뜻 보면 싸구려 호텔 방 을 연상 케 하는 감금 방 . 중국집 군 만두 만을 먹으며 8 평이 라는 제한 된 공간 에서 그 가 할 수 있는 일이 라곤 , 텔레비전 보는 게 전부 . 그렇게 1년 이 지났을 무렵 , 뉴스 를 통해 나오는 아내 의 살해 소식 . 게다가 아내 의 살인 범 으로 자신 이 지목 되고 있 음 을 알 게 된 오 대수 는 자살 을 감행 하 지만 죽는 것 조차 그 에겐 용납 되지 않는다 . 오 대수 는 복수 를 위해 체력 단련 을 비롯 , 자신 을 가둘만 한 사람 들 , 사건 들 을 모조리 기억 속 에서 꺼내 악행 의 자서전 을 기록 한다 . 한편 , 탈출 을 위해 감금 방 한쪽 구석 을 쇠 젓가락 으로 파기 도 하는데 .. 감금 15년 을 맞이 하는 해 , 마침내 사람 몸 하나 빠져나갈 만큼의 탈출 구가 생겼을 때 , 어이없게도 15년 전 납치 됐던 바로 그 장소 로 풀려나 있는 자신 을 발견 하게 된다 . 우연히 들른 일식집 에서 갑자기 정신 을 잃어버린 오 대수 는 보조 요리사 미도 집 으로 가게 되고 , 미도 는 오 대수 에게 연민 에서 시작 한 사랑 의 감정 을 키워 나가게 된다 . 한편 감금 방 에서 먹던 군 만두 에서 나온 청룡 이란 전표 하나로 찾아낸 7.5 층 감금 방 의 정체 를 찾아내고 ... 마침내 , 첫 대면 을 하는 날 복수심 으로 들끓는 대수 에게 우진 은 너무나 냉정하게 게임 을 제안 한다 . 자신 이 가둔 이유 를 5일 안 에 밝혀내면 스스로 죽어주겠다는 것 . 대수 는 이 지독한 비밀 을 풀기 위해 , 사랑 하는 연인 , 미도 를 잃지 않기 위해 5일 간의 긴박한 수수께끼 를 풀어

In [4]:
# 백터라이저 > cos유사도, eud유사도 뽑기
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer()
tfidf_m = tfidf_v.fit_transform(data_l)
tfidf_m.toarray()
from sklearn.metrics.pairwise import cosine_similarity # 코사인유사도
cos = cosine_similarity(tfidf_m, tfidf_m)
from sklearn.metrics import euclidean_distances #유클리드거리 소문자-매소드
eud = euclidean_distances(tfidf_m, tfidf_m)
cos.shape, eud.shape

((18, 18), (18, 18))

In [5]:
data.index # 이만개의 인덱
t_idx = dict(zip(data['name'], data.index )) # 집으로 결합해서 딕셔너리화   영화제목 : 인덱스
t_idx

{'007 노타임투다이': 15,
 '노트북': 1,
 '니모를찾아서': 3,
 '미션임파서블': 2,
 '반지의제왕': 6,
 '스타워즈': 17,
 '스타트렉': 12,
 '신세계': 7,
 '아저씨': 8,
 '어바웃 타임': 11,
 '올드보이': 0,
 '인터스텔라': 16,
 '장고': 10,
 '지금만나러갑니다': 14,
 '친절한금자씨': 5,
 '탑건': 4,
 '트랜스포머': 13,
 '해리포터': 9}

In [6]:
# cos 유사도로 상위5위 유사도 뽑기
# 특정 영화의 상위 유사도 영화제목으로 뽑기
def ck_s_t(t, cosine_sim = cos ):
    idx = t_idx[t]
    c_sc = list(enumerate(cosine_sim[idx])) # 넘버링 #이누머-열거
    c_sc = sorted ( c_sc, key=lambda x:x[1] , reverse=True) # reverse=True 내림차순 , 코사인유사도는 높은 것이 좋은 거니까, 디폴트는 오름차순
    m_d = c_sc[1:6] # (인덱스,유사도) (15348, 0.5258229300737997),  # 0인덱은 자기자신이니까 빼고 상위 1,2,3,4,5 위의 유사도가 높은 것을 뽑는다. 
    m_i = [i[0]for i in m_d]  # (인덱스,유사도) 를 하나 뽑아서 [0]인덱스 > 인덱스 뽑아서 리스트로 >  상위 5위까지 인덱스를 뽑았다
    return data['name'].iloc[m_i] #영화타이틀 시리즈에서 그 인덱 값들만 뽑아온다 (판다스시리즈 타입으로 뽑힌다)
pd.DataFrame(ck_s_t('올드보이'))

,name
5,친절한금자씨
11,어바웃 타임
8,아저씨
14,지금만나러갑니다
1,노트북


In [7]:
# eud로 뽑기
def ck_s_t(t, cosine_sim = eud ):
    idx = t_idx[t]
    c_sc = list(enumerate(cosine_sim[idx])) # 넘버링 #이누머-열거
    c_sc = sorted ( c_sc, key=lambda x:x[1] , reverse=True)  
    m_d = c_sc[-2:-7:-1]   # eud는 작을수록 유사도가 높으니까 뒤에서 5개를 뽑는다 -1인덱은 올드보이
    m_i = [i[0]for i in m_d]  
    return data['name'].iloc[m_i]  
pd.DataFrame(ck_s_t('올드보이'))

,name
5,친절한금자씨
11,어바웃 타임
8,아저씨
14,지금만나러갑니다
1,노트북
